# Error Analysis of the Pipeline

In [9]:
from IPython.display import Markdown, display
import json
import numpy as np


In [3]:
%run src/prediction/evaluate_veracity_verbose.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submissions/dev_mmr+gpt4o-dfewshot-tiebrk-atype.json

Question-only score (HU-meteor):             0.4566753100185953
Question-answer score (HU-meteor):           0.28764311409757226
Veracity F1 scores:
 * Supported:                                0.7368421052631579
 * Refuted:                                  0.813953488372093
 * Not Enough Evidence:                      0.2222222222222222
 * Conflicting Evidence/Cherrypicking:       0.0784313725490196
 * macro:                                    0.46286229710162313
 * acc:                                      0.708
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.688
 * Veracity scores (meteor @ 0.2):           0.538
 * Veracity scores (meteor @ 0.25):          0.414
 * Veracity scores (meteor @ 0.3):           0.296
 * Veracity scores (meteor @ 0.4):           0.134
 * Veracity scores (meteor @ 0.5):           0.05
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.2076607110651121
 * Veracity scores (

In [10]:
with open("/mnt/data/factcheck/averitec-data/data/dev.json", "r") as f:
    gold_data = json.load(f)


with open("/mnt/data/factcheck/averitec-data/data_store/submissions/dev_mmr+gpt4o-dfewshot-tiebrk-atype.json", "r") as f:
    pipeline_data = json.load(f)

with open("/mnt/data/factcheck/averitec-data/data_store/error_analysis_dev.jsonl", "r") as f:
    scores = [json.loads(line) for line in f]

In [49]:
#take random number
rand_idx = np.random.randint(len(scores))

print(scores[rand_idx])
print(gold_data[rand_idx])
print(pipeline_data[rand_idx])



scores_point = scores[rand_idx]
gold_data_point = gold_data[rand_idx]
pipeline_data_point = pipeline_data[rand_idx]

if gold_data_point["claim"] != pipeline_data_point["claim"]:
    raise ValueError("Claims are not the same!")

# Define your markdown content as a string
markdown_content = f"""
# Claim #{rand_idx} - {gold_data_point["claim"]}

Scores Q: **{scores_point["q_score"]:.2f}** Q+A: **{scores_point["qa_score"]:.2f}**

Gold Label: **{gold_data_point["label"]}** | Predicted Label: **{pipeline_data_point["pred_label"]}**

## Gold Data
"""

for key, value in gold_data_point.items():
    if key == "claim":
        continue
    elif key == "label":
        continue
    elif key == "questions":
        markdown_content += f"**{key}:**\n"
        for question in value:
            markdown_content += f"  - **question:** {question['question']}\n"
            for a in question["answers"]:
                markdown_content += f"    - **answer:** {a['answer']}, **asnwer_type:** {a['answer_type']}, **source_url:** {a['source_url']}, **source_medium:** {a['source_medium']}, **cached_source_url:** {a['cached_source_url']}\n"
                if a["answer_type"] == "Boolean":
                    markdown_content += f"    - **boolean explanation:** {a['boolean_explanation']}\n"
    else:
        markdown_content += f"**{key}:** {value}\n\n"

markdown_content += "\n## Pipeline Data\n"

for key, value in pipeline_data_point.items():
    if key == "pred_label":
        continue
    elif key == "claim":
        continue
    elif key == "claim_id":
        continue
    elif key == "evidence":
        markdown_content += f"**{key}:**\n"
        for question in value:
            markdown_content += f"  - **question:** {question['question']}, **answer:** {question['answer']}, **answer_type:** {question['answer_type']}\n\n    - **url:** {question['url']}\n\n    - **scraped text:** {question['scraped_text']}\n"
            
    else:
        markdown_content += f"**{key}:** {value}\n\n"

# Display the markdown
display(Markdown(markdown_content))

{'claim_id': 315, 'q_score': 0.4013356747552149, 'qa_score': 0.12777529553138023}
{'claim': 'The fastest Supreme Court justice ever confirmed in the U.S. was 47 days.', 'required_reannotation': False, 'label': 'Refuted', 'justification': 'This claim is refuted. According to the QA pairs, the quickest time a Supreme Court justice nomination has been confirmed in the United States is 19 days. John Paul Steven leads the fastest confirmed Supreme Court confirmation (19 days), followed by Amy Coney Barrett with 27 days. The claim states that the "fastest Supreme Court justice ever confirmed in the U.S. was 47 days". This is not proven in the QA pairs. Therefore, this claim is refuted. ', 'claim_date': '18-9-2020', 'speaker': 'Joe Biden', 'original_claim_url': None, 'fact_checking_article': 'https://web.archive.org/web/20210711055417/https://www.factcheck.org/2020/09/bidens-false-and-exaggerated-supreme-court-claims/', 'reporting_source': None, 'location_ISO_code': 'US', 'claim_types': ['Num


# Claim #315 - The fastest Supreme Court justice ever confirmed in the U.S. was 47 days.

Scores Q: **0.40** Q+A: **0.13**

Gold Label: **Refuted** | Predicted Label: **Refuted**

## Gold Data
**required_reannotation:** False

**justification:** This claim is refuted. According to the QA pairs, the quickest time a Supreme Court justice nomination has been confirmed in the United States is 19 days. John Paul Steven leads the fastest confirmed Supreme Court confirmation (19 days), followed by Amy Coney Barrett with 27 days. The claim states that the "fastest Supreme Court justice ever confirmed in the U.S. was 47 days". This is not proven in the QA pairs. Therefore, this claim is refuted. 

**claim_date:** 18-9-2020

**speaker:** Joe Biden

**original_claim_url:** None

**fact_checking_article:** https://web.archive.org/web/20210711055417/https://www.factcheck.org/2020/09/bidens-false-and-exaggerated-supreme-court-claims/

**reporting_source:** None

**location_ISO_code:** US

**claim_types:** ['Numerical Claim']

**fact_checking_strategies:** ['Numerical Comparison']

**questions:**
  - **question:** What is the quickest time a Supreme Court justice nomination has been confirmed in the United States?
    - **answer:** John Paul Stevens waited the fewest number of days (19)—followed by the most recent nominee to the Court, Amy Coney Barrett (27).61, **asnwer_type:** Extractive, **source_url:** https://web.archive.org/web/20210710011712/https://www.fas.org/sgp/crs/misc/R44234.pdf, **source_medium:** PDF, **cached_source_url:** https://web.archive.org/web/20210710011712/https://www.fas.org/sgp/crs/misc/R44234.pdf
  - **question:** What is the average number of days between a nomination for a Supreme Court justice and the final Senate vote?
    - **answer:** Overall, the average number of days from nomination to final Senate vote is 68.2 days (or approximately 2.2 months), while the median is 69.0 days.62
Of the 9 Justices currently serving on the Court, the average number of days from nomination to
final Senate vote is 72.1 days (or approximately 2.4 months), while the median is 73.0 days.
Among the current Justices, Amy Coney Barrett waited the fewest number of days from
nomination to confirmation (27), while Clarence Thomas waited the greatest number of days
(99)., **asnwer_type:** Extractive, **source_url:** https://web.archive.org/web/20210710011712/https://www.fas.org/sgp/crs/misc/R44234.pdf, **source_medium:** PDF, **cached_source_url:** https://web.archive.org/web/20210710011712/https://www.fas.org/sgp/crs/misc/R44234.pdf
**cached_original_claim_url:** None


## Pipeline Data
**evidence:**
  - **question:** Who was the fastest Supreme Court justice ever confirmed in the U.S.?, **answer:** The fastest Supreme Court justice ever confirmed in the U.S. was Ruth Bader Ginsburg, who was confirmed in 50 days., **answer_type:** Extractive

    - **url:** https://www.cbsnews.com/news/sotomayor-confirmed-in-below-average-time/

    - **scraped text:** 
"I want to thank the Senate Judiciary Committee, particularly its Chairman, Senator Leahy -- as well as its Ranking Member, Senator Sessions -- for giving Judge Sotomayor a thorough and civil hearing. And I thank them for doing so in a timely manner so that she can be fully prepared to take her seat when the Court's work begins this September," Mr. Obama said. According to CBS News White House correspondent Mark Knoller, that it has been two months and eleven days since Mr. Obama nominated Sotomayor. CBS News White House produer Robert Hendin points out that this means Sotomayor's waiting period was identical to that of Chief Justice John Roberts, with both confirmations taking 72 days from nomination to floor vote. Both Sotomayor's and Roberts' appointments took less than the 81 day average for recent nominees. According to Hendin, Sotomayor's predecessor, David Souter, was confirmed in 71 days. Ruth Bader Ginsburg was appointed in a short 50 days. But Current Supreme Court justices Samuel Alito and Antonin Scalia's confirmations took 92 days, as did the confirmation of former Chief Justice William Rehnquist. Clarence Thomas tops the list with the longest period between nomination and the floor vote amongst presiding members. 

  - **question:** How long did it take for Ruth Bader Ginsburg to be confirmed?, **answer:** 50 days, **answer_type:** Extractive

    - **url:** https://www.cbsnews.com/news/sotomayor-confirmed-in-below-average-time/

    - **scraped text:** 
"I want to thank the Senate Judiciary Committee, particularly its Chairman, Senator Leahy -- as well as its Ranking Member, Senator Sessions -- for giving Judge Sotomayor a thorough and civil hearing. And I thank them for doing so in a timely manner so that she can be fully prepared to take her seat when the Court's work begins this September," Mr. Obama said. According to CBS News White House correspondent Mark Knoller, that it has been two months and eleven days since Mr. Obama nominated Sotomayor. CBS News White House produer Robert Hendin points out that this means Sotomayor's waiting period was identical to that of Chief Justice John Roberts, with both confirmations taking 72 days from nomination to floor vote. Both Sotomayor's and Roberts' appointments took less than the 81 day average for recent nominees. According to Hendin, Sotomayor's predecessor, David Souter, was confirmed in 71 days. Ruth Bader Ginsburg was appointed in a short 50 days. But Current Supreme Court justices Samuel Alito and Antonin Scalia's confirmations took 92 days, as did the confirmation of former Chief Justice William Rehnquist. Clarence Thomas tops the list with the longest period between nomination and the floor vote amongst presiding members. 

  - **question:** What is the average time for recent Supreme Court nominees to be confirmed?, **answer:** 81 days, **answer_type:** Extractive

    - **url:** https://www.cbsnews.com/news/sotomayor-confirmed-in-below-average-time/

    - **scraped text:** 
"I want to thank the Senate Judiciary Committee, particularly its Chairman, Senator Leahy -- as well as its Ranking Member, Senator Sessions -- for giving Judge Sotomayor a thorough and civil hearing. And I thank them for doing so in a timely manner so that she can be fully prepared to take her seat when the Court's work begins this September," Mr. Obama said. According to CBS News White House correspondent Mark Knoller, that it has been two months and eleven days since Mr. Obama nominated Sotomayor. CBS News White House produer Robert Hendin points out that this means Sotomayor's waiting period was identical to that of Chief Justice John Roberts, with both confirmations taking 72 days from nomination to floor vote. Both Sotomayor's and Roberts' appointments took less than the 81 day average for recent nominees. According to Hendin, Sotomayor's predecessor, David Souter, was confirmed in 71 days. Ruth Bader Ginsburg was appointed in a short 50 days. But Current Supreme Court justices Samuel Alito and Antonin Scalia's confirmations took 92 days, as did the confirmation of former Chief Justice William Rehnquist. Clarence Thomas tops the list with the longest period between nomination and the floor vote amongst presiding members. 

  - **question:** How long did it take for Sonia Sotomayor to be confirmed?, **answer:** 72 days, **answer_type:** Extractive

    - **url:** https://www.cbsnews.com/news/sotomayor-confirmed-in-below-average-time/

    - **scraped text:** 
"I want to thank the Senate Judiciary Committee, particularly its Chairman, Senator Leahy -- as well as its Ranking Member, Senator Sessions -- for giving Judge Sotomayor a thorough and civil hearing. And I thank them for doing so in a timely manner so that she can be fully prepared to take her seat when the Court's work begins this September," Mr. Obama said. According to CBS News White House correspondent Mark Knoller, that it has been two months and eleven days since Mr. Obama nominated Sotomayor. CBS News White House produer Robert Hendin points out that this means Sotomayor's waiting period was identical to that of Chief Justice John Roberts, with both confirmations taking 72 days from nomination to floor vote. Both Sotomayor's and Roberts' appointments took less than the 81 day average for recent nominees. According to Hendin, Sotomayor's predecessor, David Souter, was confirmed in 71 days. Ruth Bader Ginsburg was appointed in a short 50 days. But Current Supreme Court justices Samuel Alito and Antonin Scalia's confirmations took 92 days, as did the confirmation of former Chief Justice William Rehnquist. Clarence Thomas tops the list with the longest period between nomination and the floor vote amongst presiding members. 

  - **question:** How long did it take for John Roberts to be confirmed?, **answer:** 72 days, **answer_type:** Extractive

    - **url:** https://www.cbsnews.com/news/sotomayor-confirmed-in-below-average-time/

    - **scraped text:** 
"I want to thank the Senate Judiciary Committee, particularly its Chairman, Senator Leahy -- as well as its Ranking Member, Senator Sessions -- for giving Judge Sotomayor a thorough and civil hearing. And I thank them for doing so in a timely manner so that she can be fully prepared to take her seat when the Court's work begins this September," Mr. Obama said. According to CBS News White House correspondent Mark Knoller, that it has been two months and eleven days since Mr. Obama nominated Sotomayor. CBS News White House produer Robert Hendin points out that this means Sotomayor's waiting period was identical to that of Chief Justice John Roberts, with both confirmations taking 72 days from nomination to floor vote. Both Sotomayor's and Roberts' appointments took less than the 81 day average for recent nominees. According to Hendin, Sotomayor's predecessor, David Souter, was confirmed in 71 days. Ruth Bader Ginsburg was appointed in a short 50 days. But Current Supreme Court justices Samuel Alito and Antonin Scalia's confirmations took 92 days, as did the confirmation of former Chief Justice William Rehnquist. Clarence Thomas tops the list with the longest period between nomination and the floor vote amongst presiding members. 

  - **question:** How long did it take for Samuel Alito to be confirmed?, **answer:** 92 days, **answer_type:** Extractive

    - **url:** https://www.cbsnews.com/news/sotomayor-confirmed-in-below-average-time/

    - **scraped text:** 
"I want to thank the Senate Judiciary Committee, particularly its Chairman, Senator Leahy -- as well as its Ranking Member, Senator Sessions -- for giving Judge Sotomayor a thorough and civil hearing. And I thank them for doing so in a timely manner so that she can be fully prepared to take her seat when the Court's work begins this September," Mr. Obama said. According to CBS News White House correspondent Mark Knoller, that it has been two months and eleven days since Mr. Obama nominated Sotomayor. CBS News White House produer Robert Hendin points out that this means Sotomayor's waiting period was identical to that of Chief Justice John Roberts, with both confirmations taking 72 days from nomination to floor vote. Both Sotomayor's and Roberts' appointments took less than the 81 day average for recent nominees. According to Hendin, Sotomayor's predecessor, David Souter, was confirmed in 71 days. Ruth Bader Ginsburg was appointed in a short 50 days. But Current Supreme Court justices Samuel Alito and Antonin Scalia's confirmations took 92 days, as did the confirmation of former Chief Justice William Rehnquist. Clarence Thomas tops the list with the longest period between nomination and the floor vote amongst presiding members. 

  - **question:** How long did it take for Antonin Scalia to be confirmed?, **answer:** 92 days, **answer_type:** Extractive

    - **url:** https://www.cbsnews.com/news/sotomayor-confirmed-in-below-average-time/

    - **scraped text:** 
"I want to thank the Senate Judiciary Committee, particularly its Chairman, Senator Leahy -- as well as its Ranking Member, Senator Sessions -- for giving Judge Sotomayor a thorough and civil hearing. And I thank them for doing so in a timely manner so that she can be fully prepared to take her seat when the Court's work begins this September," Mr. Obama said. According to CBS News White House correspondent Mark Knoller, that it has been two months and eleven days since Mr. Obama nominated Sotomayor. CBS News White House produer Robert Hendin points out that this means Sotomayor's waiting period was identical to that of Chief Justice John Roberts, with both confirmations taking 72 days from nomination to floor vote. Both Sotomayor's and Roberts' appointments took less than the 81 day average for recent nominees. According to Hendin, Sotomayor's predecessor, David Souter, was confirmed in 71 days. Ruth Bader Ginsburg was appointed in a short 50 days. But Current Supreme Court justices Samuel Alito and Antonin Scalia's confirmations took 92 days, as did the confirmation of former Chief Justice William Rehnquist. Clarence Thomas tops the list with the longest period between nomination and the floor vote amongst presiding members. 

  - **question:** How long did it take for Clarence Thomas to be confirmed?, **answer:** 99 days, **answer_type:** Extractive

    - **url:** https://www.cbsnews.com/news/sotomayor-confirmed-in-below-average-time/

    - **scraped text:** 
"I want to thank the Senate Judiciary Committee, particularly its Chairman, Senator Leahy -- as well as its Ranking Member, Senator Sessions -- for giving Judge Sotomayor a thorough and civil hearing. And I thank them for doing so in a timely manner so that she can be fully prepared to take her seat when the Court's work begins this September," Mr. Obama said. According to CBS News White House correspondent Mark Knoller, that it has been two months and eleven days since Mr. Obama nominated Sotomayor. CBS News White House produer Robert Hendin points out that this means Sotomayor's waiting period was identical to that of Chief Justice John Roberts, with both confirmations taking 72 days from nomination to floor vote. Both Sotomayor's and Roberts' appointments took less than the 81 day average for recent nominees. According to Hendin, Sotomayor's predecessor, David Souter, was confirmed in 71 days. Ruth Bader Ginsburg was appointed in a short 50 days. But Current Supreme Court justices Samuel Alito and Antonin Scalia's confirmations took 92 days, as did the confirmation of former Chief Justice William Rehnquist. Clarence Thomas tops the list with the longest period between nomination and the floor vote amongst presiding members. 

  - **question:** What is the longest time before confirmation in the past three decades?, **answer:** 99 days, for Justice Thomas, **answer_type:** Extractive

    - **url:** https://obamawhitehouse.archives.gov/scotus

    - **scraped text:** 
The confirmation of a Supreme Court Justice is a solemn responsibility that the President and the Senate share under the U.S. Constitution. Throughout history, members of both parties in Congress and in the White House have done their jobs so that the Judicial Branch can do its own. Failing to do so would be an unprecedented dereliction of duty. The Senate has almost a full year to consider and confirm a nominee. In fact, since 1975, the average time from nomination to confirmation is 67 days. The longest time before confirmation in the past three decades was 99 days, for Justice Thomas, and the last four Justices, spanning two Administrations, were confirmed in an average of 75 days. Six Justices have been confirmed in a presidential election year since 1900. Every nominee who was not withdrawn has received a vote within 125 days of nomination. It will be harmful and create unsustainable uncertainty if Congress fails to act on the President's nominee. Here's a look at where President Obama's Supreme Court nominee stands in the confirmation process compared to those of recent nominees under three different administrations. Q: Has Judge Garland issued opinions hostile to gun owners? A: No. Chief Judge Garland has never issued or joined an opinion involving the Second Amendment. Some right-wing special interest groups are spending millions of dollars trying to distort Judge Garland’s record by misrepresenting his votes in two cases. Independent fact-checkers have already deemed those attacks “false,” but let’s look at the facts. In 2008, a D.C. Circuit panel that did not include Chief Judge Garland struck down parts of a Washington, D.C. gun control law in Parker v. District of Columbia, a decision that was eventually affirmed by the Supreme Court in District of Columbia v. Heller. Chief Judge Garland joined one of President George H.W. Bush’s appointees and two other judges in voting to grant rehearing by the full court. 
A vote for rehearing by the full court is simply a request to have more judges take a look at the legal matter. Judge Garland’s vote to request rehearing does not indicate anything about his views on the Second Amendment. The D.C. Circuit’s rules say that rehearing by the full court is only appropriate in certain circumstances, such as where the case involves a question of exceptional importance. As Chief Justice Roberts explained at his confirmation hearing, there are many reasons judges vote for rehearing, including that a case involves important questions that the court “ought to look at” more closely. This case was the first federal court decision to strike down a statute on Second Amendment grounds in decades. Chief Judge Garland was also on the D.C. Circuit when it denied full court rehearing in Heller II, a September 2015 panel decision invalidating various portions of a new D.C. gun control law under the Second Amendment. The second case, the 2000 decision in NRA v. Reno, does not reflect any views on the Second Amendment, but rather Chief Judge Garland’s careful case-by-case approach to judging. This case was about a Justice Department regulation that allowed the Department to temporarily retain certain data generated during background checks of prospective firearms purchasers that are required by federal law. The Department kept the data for a limited period to review the background system to ensure accuracy and privacy. Garland voted with the majority to uphold the regulation, in an opinion that concluded that the Department had reasonably interpreted the Brady Act to permit retaining the data temporarily to conduct audits. The D.C. Circuit emphasized that the audit records did not represent a “system for the registration of firearms, firearms owners, or firearms transactions” and that the creation of any such system would be barred by statute. The case was about administrative law and did not mention the Second Amendment. 
  - **question:** How long did it take for Elena Kagan to be confirmed?, **answer:** 87 days, **answer_type:** Extractive

    - **url:** https://www.npr.org/2016/02/14/466723547/7-things-to-know-about-presidential-appointments-to-the-supreme-court

    - **scraped text:** Rutledge is the earliest rejection of a nominee (by a 14-10 vote and also the only recess appointment to be rejected). In 1811, Alexander Wolcott was the fastest non-justice to be rejected. It took just nine days for the Senate to dismiss Madison's nominee. Longest waits for presidential nominees to the Supreme Court (from receipt in Congress to result): 125 days — Louis D. Brandeis (Wilson), confirmed 47-22, June 1, 1916 (19 public hearings, first was 12 days after nomination) 108 days — Robert H. Bork (Reagan), rejected, 58-42, Oct. 23, 1987 (12 public hearings, first was 70 days after nomination) 108 days — Potter Stewart (Eisenhower), confirmed 70-17, May 5, 1959, after recess appointment 100 days — Abe Fortas (Johnson), withdrawn after being nominated to be chief justice, Oct. 4, 1968 (11 public hearings, first was 15 days after nomination) 100 days — Homer Thornberry (Johnson), withdrawn, Oct. 4, 1968 (11 public hearings, first 15 days after nomination) 99 days — Clarence Thomas (H.W. Bush), confirmed 52-48, Oct. 15, 1991 (11 public hearings, first was 64 days after nomination) 96 days — Reuben H. Walworth (Tyler), withdrawn, June 14, 1844 92 days — Clement F. Haynsworth Jr. (Nixon), rejected 55-45, Nov. 21, 1969 (8 public hearings, first was 26 days after nomination) 89 days — William H. Rehnquist (Reagan), confirmed to be chief justice 65-33, Sept. 17, 1986 (4 public hearings, first was 39 days after nomination) 87 days — Elena Kagan (Obama), confirmed 63-37, Aug. 5, 2010 (4 public hearings, first was 49 days after nomination) 85 days — Antonin Scalia (Reagan), confirmed 98-0, Sept. 17, 1986 (2 public hearings, first was 42 days after nomination) 82 days — Samuel Alito (W. Bush), confirmed 58-42, Jan. 31, 2006 (5 public hearings, first was 60 days after nomination) 79 days — Melville W. Fuller (Cleveland), confirmed 41-20, July 20, 1888 (no recorded hearings) 79 days — George Harrold Carswell (Nixon), rejected 51-45, April 8, 1970 (5 hearings, first was 8 days after nomination) 
78 days — Thurgood Marshall (Johnson), confirmed 69-11, Aug. 3, 1967 (5 public hearings) An earlier version of this post incorrectly identified the last person nominated and confirmed to the Supreme Court in a presidential election year as Stanley Reed in 1938. 

